In [80]:
from typing import List, Optional
import requests
import geopandas as gpd
from shapely.geometry import Polygon
import folium

In [87]:
class MapTiles:
    """
    Fetch the footprints for given satellite scenes.

    This class handles the odata queries for the satellite scenes
    and builds an output geospatial file.
    """

    def __init__(
        self,
        input_products: Optional[list] = None,
    ):
        if isinstance(input_products, str):
            self.input_products = [input_products]
        elif isinstance(input_products, list):
            self.input_products = input_products
        else:
            raise ValueError("input_products must be a list or a string")
        self.catalog_entries = None
        self.no_data = []

    def query_catalog(self) -> List:
        if not self.input_products:
            raise ValueError("No input data provided.")
        json_response = []
        for product in self.input_products:
            try:
                json = requests.get(
                    f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=startswith(Name, '{product}')"
                ).json()
                if json:
                    json_response.append(json["value"][0])
                else:
                    self.no_data.append(product)
            except Exception as e:
                raise ValueError(f"Failed to query the catalogue: {e}")

            gdf = gpd.GeoDataFrame(json_response)
            gdf.set_geometry(
                gdf["GeoFootprint"].apply(lambda x: Polygon(x["coordinates"][0])),
                inplace=True,
            )
            gdf.set_crs(epsg=4326, inplace=True)
            self.catalog_entries = gdf

    def plot_footprints(self):
        if self.catalog_entries is None:
            raise ValueError("No catalog entries found.")
        # Explore the GeoDataFrame, visualizing the 'area' column with a specified colormap
        reprojected_gdf = self.catalog_entries.to_crs(epsg=4326)
        m = folium.Map()
        fit_bounds = reprojected_gdf.total_bounds.tolist()
        m.fit_bounds([[fit_bounds[1], fit_bounds[0]], [fit_bounds[3], fit_bounds[2]]])
        folium.GeoJson(reprojected_gdf).add_to(m)

        return m

In [88]:
# Sentinel 2
s2_single_scene = "S2B_MSIL1C_20250213T100029_N0511_R122_T33TVN_20250213T115442"

mt = MapTiles(s2_single_scene)
print(mt.input_products)

mt.query_catalog()

mt.plot_footprints()

['S2B_MSIL1C_20250213T100029_N0511_R122_T33TVN_20250213T115442']


In [89]:
s2_multiple_scenes = [
    "S2B_MSIL1C_20250213T100029_N0511_R122_T33TVN_20250213T115442",
    "S2B_MSIL2A_20250206T101109_N0511_R022_T32TQT_20250206T122414",
]

mt2 = MapTiles(s2_multiple_scenes)
mt2.query_catalog()

print(mt2.input_products)
print(mt2.catalog_entries)
print(mt2.no_data)
mt2.plot_footprints()

['S2B_MSIL1C_20250213T100029_N0511_R122_T33TVN_20250213T115442', 'S2B_MSIL2A_20250206T101109_N0511_R022_T32TQT_20250206T122414']
    @odata.mediaContentType                                    Id  \
0  application/octet-stream  86edd327-bbe7-457a-8ec1-07b19e8e3654   
1  application/octet-stream  cde8f7e9-276d-41b2-a19d-73ecad583124   

                                                Name  \
0  S2B_MSIL1C_20250213T100029_N0511_R122_T33TVN_2...   
1  S2B_MSIL2A_20250206T101109_N0511_R022_T32TQT_2...   

                ContentType  ContentLength                   OriginDate  \
0  application/octet-stream      878344227  2025-02-13T12:24:37.000000Z   
1  application/octet-stream     1201759031  2025-02-06T13:07:57.000000Z   

               PublicationDate             ModificationDate  Online  \
0  2025-02-13T12:43:17.547157Z  2025-02-14T09:16:31.534511Z    True   
1  2025-02-06T13:22:18.271274Z  2025-02-06T13:30:58.561823Z    True   

                  EvictionDate  \
0  9999-12-31T23:59: